In [1]:
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
from rnn_layers import PredictTime
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device: ", device)

df = get_data()

Using device:  cuda


In [2]:

def make_data(df, device):

    x_train, y_train = [], []
    prev = None

    for row in df:
        if prev is None:
            prev = row
        x_train.append(prev)
        y_train.append(row)
        prev = row
    print(x_train[0].size())
    return torch.stack(x_train).float().to(device),torch.stack(y_train).float().to(device)

make_data(get_binary_values(get_data()), device)



torch.Size([1])


(tensor([[1.],
         [1.],
         [0.],
         ...,
         [1.],
         [0.],
         [1.]], device='cuda:0'),
 tensor([[1.],
         [0.],
         [1.],
         ...,
         [0.],
         [1.],
         [1.]], device='cuda:0'))

In [ ]:
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from IPython.display import clear_output
import torch.nn as nn
import random

batch_size = 100
x_d, y_d = make_data_scalar(get_data(), device)

print(y_d.size(), x_d.size())

model = PredictTime(input_size=x_d[0].size()[0],
                    output_size=y_d[0].size()[0],
                    hidden_layers=5,
                    hidden_size=25, device=device).to(device)
optimizer = optim.Adam(model.parameters())

loss = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(x_d,y_d), batch_size=batch_size)
epochs = 5000
for e in range(epochs):
    model.train()
    #print(next(iter(loader)))
    model.clean_state()
    res = []
    
    
    if random.random() < e/epochs:
        model.teacher_forcing = False
    else:
        model.teacher_forcing = True
    
    for x, y in loader:
        
        if random.random() < 0.3:
            continue
        model.init_state()
        
        
        y_pred = model(x)
        l = loss(y_pred, y)
        res.append(l)
        #print(y_pred, y)
    
    
    l = res[0]
    for i in res[1:]:
        l += i
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if e % 10 != 0:
        continue
    #clear_output(wait=True)
    sum_loss = 0
    #print(list(model.parameters())[-1])

    for x, y in loader:
        model.eval()
        with torch.no_grad():
            y_pred = model(x)
            sum_loss += np.sqrt(loss(y_pred, y).cpu())
    
    print("Epoch %d Loss %.4f" % (e, sum_loss))
    
    
#for d in df.values:

Max value:  1000
torch.Size([8152, 1]) torch.Size([8152, 1])
Epoch 0 Loss 35.7039
Epoch 10 Loss 34.3000
Epoch 20 Loss 31.9188
Epoch 30 Loss 29.6306
Epoch 40 Loss 29.6694
Epoch 50 Loss 29.6490
Epoch 60 Loss 29.6764
Epoch 70 Loss 29.6342
Epoch 80 Loss 29.6283
Epoch 90 Loss 29.6335
Epoch 100 Loss 29.6336
Epoch 110 Loss 29.6304
Epoch 120 Loss 29.6306
Epoch 130 Loss 29.6341
Epoch 140 Loss 29.6443
Epoch 150 Loss 29.6340
Epoch 160 Loss 29.6331
Epoch 170 Loss 29.6302
Epoch 180 Loss 29.6318
Epoch 190 Loss 29.6290
Epoch 200 Loss 29.6291
Epoch 210 Loss 29.6330
Epoch 220 Loss 29.6382
Epoch 230 Loss 29.6463
Epoch 240 Loss 29.6318
Epoch 250 Loss 29.6351
Epoch 260 Loss 29.6273
Epoch 270 Loss 29.6346
Epoch 280 Loss 29.6323
Epoch 290 Loss 29.6302
Epoch 300 Loss 29.6271
Epoch 310 Loss 29.6278
Epoch 320 Loss 29.6196
Epoch 330 Loss 29.6119
Epoch 340 Loss 29.5927
Epoch 350 Loss 29.5242
Epoch 360 Loss 29.3916
Epoch 370 Loss 29.4091
Epoch 380 Loss 29.4361
Epoch 390 Loss 29.6254
Epoch 400 Loss 29.3537
Epoch 4

In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    

    fig, ax = plt.subplots(2)
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result when feeding correct values as input")
    fig.savefig("teacher_forcing.png")


amount = 1000
eval_model(x_d,y_d,model)


In [ ]:
import matplotlib.pyplot as plt

def eval_model(x,y,m):
    df = get_data() 
    maxtime = df.max()[1]

    res = []
    m.eval()
    m.clean_state()
    count  = 1
    prev = x[0][0]
    for i in x:
        t = torch.tensor([[prev]]).to(device)
        prev = i
        val = m(t)
        prev = val
        count += 1
        res.append(val.detach().cpu()[0])
    
    loss = nn.MSELoss()
    print(np.sqrt(loss(torch.tensor(res).to(device),y.squeeze()).cpu()))
    
    fig, ax = plt.subplots(2)
    
    ax[0].plot(range(1,51), res[:50])
    ax[0].plot( range(1,51), y[:50].cpu())

    ax[1].plot(range(2000,2050), res[2000:2050])
    
    ax[1].plot(range(2000,2050), y[2000:2050].cpu() )
    fig.suptitle("Result without feeding correct values as input")
    fig.savefig("teacher_forcing_not_correct.png")
    
amount = 1000
eval_model(x_d,y_d,model)
